In [ ]:
"""squeezed_cavity_Finesse.ipynb
James Gardner 2020

Finesse model for the non-linear element in a cavity.
Produces the plot and values used to compare against the analytics of Aim 1 of the ASC goals.

To execute, run all cells.
"""

In [ ]:
# import packages
import numpy as np
import matplotlib.pyplot as plt  
%matplotlib inline
# from IPython.display import display, HTML

from pykat import finesse
from pykat.commands import *
pykat.init_pykat_plotting(dpi=90)

In [ ]:
# font sizes for plotting later
SMALL_SIZE = 18
MEDIUM_SIZE = 20
BIGGER_SIZE = 22

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE-2)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
# base model that holds the components, we run detection models on top of this
basekat = finesse.kat()
basekat.verbose = False

basecode = """
# no spaces in model, except inside cavity

l laser 1 0 0 nbs1 # power = 1W
# beamsplitter
bs BS 0.5 0.5 0 45 nbs1 nbs2 nbs3 nbs4 # phi = 0

# s is for space
s sdbs1 0 nbs4 ndbs1 # zero length space
# dbs is used as a Faraday isolator to re-direct beams
dbs isol ndbs1 ndbs2 ndbs3 ndbs4  # dbs doesn't req R T phi alpha
s sdbs2 0 ndbs2 nc0 # zero length space

# cavity, m is for mirror, nle is the non-linear element
m m1 0.9 0.1 0 nc0 nc1 # R = 0.9, T = 0.1, phi = 0
s sc1 0.5 nc1 nc2 # total cavity L = 1m
nle nle1 0.05 0 nc2 nc3 # r = 0.05, sqz angle = 0
s sc2 0.5 nc3 nc4 # total cavity L = 1m
m m2 1 0 0 nc4 nc5 # R = 1, T = 0, phi = 0

# homodyne detector and quantum noise
qhd qn 180 nbs2 nbs3
fsig noise 1 # noise at 1 Hz

# by default noise enters through every open port, lossy optic, or squeezer
# -- Quantum noise inputs --
#   |- %10-.10s - laser Node LASER NOISE
#   |- %10-.10s - m2 Node OPEN PORT
#   |- %10-.10s - BS Node OPEN PORT
#   |- %10-.10s - BS Node OPEN PORT
#   |- %10-.10s - isol Node OPEN PORT
#   |- %10-.10s - isol Node OPEN PORT
# can instead specify exactly where through "vacuum" command
# vacuum isol laser
#printnoises
"""
basekat.parse(basecode)

In [ ]:
# check that the model built correctly
print((basekat.components, basekat.detectors, basekat.commands))

In [ ]:
# vary the squeezer strength
# we want to find the threshold value first
kat1 = deepcopy(basekat)
kat1.verbose = False
kat1code = """
xaxis nle1 r lin -1 1 10000 # to spot threshold
#yaxis abs:deg
"""

kat1.parse(kat1code)

# print((kat1.components, kat1.detectors, kat1.commands))
out1 = kat1.run()
print(out1.stdout)

In [ ]:
# plot quantum noise against squeezer parameter
# out1.plot(detectors=['qn'], title="homodyne intensity vs squeezer parameter")
# # filename='pykat_qhd_vs_r.pdf'
# out1.plot(title="homodyne intensity vs squeezer parameter",
#           ylim=(0, 0.5*1e-8))

# checking limits against HomI(0)
homI_zero = out1.y[np.searchsorted(out1.x, 0)][0]
print(out1.y[0], homI_zero, out1.y[-1])

# finding threshold
print(out1.x[out1.y.argmin()], out1.x[out1.y.argmax()])
# rThresh
rT = 0.1144

# fig, ax = plt.subplots(figsize=(8,6))
# ax.plot(out1.x, 20*np.log10(out1.y/homI_zero))
# ax.set_ylim(0, 5)
# ax.set(title="relative homodyne intensity vs squeezer parameter",
#        xlabel="squeezer parameter, r",
#        ylabel="abs of relative noise")
# plt.show()

# importing results from analytics
analytics_data = np.genfromtxt("squeezed_cavity_analytics_homodyne_intensity_vs_sqz_param.csv",
                               delimiter=",")
# plt.plot(10*np.log10(np.exp(analytics_data[:,0])), analytics_data[:,1])

fig, ax = plt.subplots(figsize=(8,6))
ax.plot(out1.x, out1.y/homI_zero, label="Finesse nle")
ax.plot(10*np.log10(np.exp(analytics_data[:,0])), analytics_data[:,1], label="analytics")
ax.set(title="relative homodyne intensity vs squeezer parameter",
       xlabel="squeezer parameter, r / \"dB\" (as $10\,\log_{10}$)",
       ylabel="abs of relative noise", ylim=(0,4.995))
ax.tick_params(axis='both', labelsize=16)
plt.title("relative homodyne intensity vs squeezer parameter", fontsize=17)
ax.legend(loc="upper left")

fig.tight_layout()
fig.savefig("squeezed_cavity_relative_qhd_vs_r_comparison.pdf")
plt.show()

In [ ]:
# # doing it all with sq, the existing squeezer, to compare
# kat2 = finesse.kat()
# kat2.verbose = False

# kat2code = """
# l laser 1 0 0 nbs1 # power = 1W
# bs BS 0.5 0.5 0 45 nbs1 nbs2 nbs3 nbs4 # phi = 0

# s sdbs1 0 nbs4 ndbs1 # zero length space
# dbs isol ndbs1 ndbs2 ndbs3 ndbs4  # dbs doesn't req R T phi alpha
# s sdbs2 0 ndbs2 nc0 # zero length space

# sq sqz1 0 0.1 90 nc0

# # homodyne detector and quantum noise
# qhd qn 180 nbs2 nbs3
# fsig noise 1 # noise at 1 Hz

# xaxis sqz1 r lin 0 1 10000 # to spot threshold
# #yaxis abs:deg
# """
# kat2.parse(kat2code)

# out2 = kat2.run()

# # filename='sqz_cavity_Finesse_qhd_vs_r_for_sq.pdf'
# out2.plot(title="homodyne intensity vs squeezer parameter",
#           ylim=(0, 0.5*1e-8))

In [ ]:
# now, model a squeezed cavity again, but need a way to set r
# this here is a hack: hard code everything again
hackbase = finesse.kat()
hackbase.verbose = False

hackbasecode = """
l laser 1 0 0 nbs1 # power = 1W
bs BS 0.5 0.5 0 45 nbs1 nbs2 nbs3 nbs4 # phi = 0

s sdbs1 0 nbs4 ndbs1 # zero length space
dbs isol ndbs1 ndbs2 ndbs3 ndbs4  # dbs doesn't req R T phi alpha
s sdbs2 0 ndbs2 nc0 # zero length space

m m1 0.9 0.1 0 nc0 nc1 # R = 0.9, T = 0.1, phi = 0
s sc1 0.5 nc1 nc2 # total cavity L = 1m
s sc2 0.5 nc3 nc4 # total cavity L = 1m
m m2 1 0 0 nc4 nc5 # R = 1, T = 0, phi = 0

qhd qn 180 nbs2 nbs3
fsig noise 1 # noise at 1 Hz

xaxis noise f lin 0 1G 10000
"""
hackbase.parse(hackbasecode)

In [ ]:
# these functions hard set the r parameter
def dB_hacksqz(sqz_r_dB):
    """given a squeezer parameter in dB, return output of model"""
    hackkat = deepcopy(hackbase)
    
    hackcode = """
    nle nle1 {0} 0 nc2 nc3 # sqz angle = 0
    """.format(sqz_r_dB)

    hackkat.parse(hackcode)

    return hackkat.run()

def dB_rel_hacksqz(sqz_r):
    """given a squeezer parameter, return output and relative y scale"""
    zeroreference = dB_hacksqz(0).y
    out = dB_hacksqz(sqz_r)
    return out, out.y/zeroreference

def negative_test_dB_hacksqz(sqz_r_dB):
    """given a squeezer parameter in dB, return output of model"""
    hackkat = deepcopy(hackbase)
    
    hackcode = """
    nle nle1 {0} 90 nc2 nc3 # sqz angle = 90
    """.format(sqz_r_dB)

    hackkat.parse(hackcode)

    return hackkat.run()

def negative_test_dB_rel_hacksqz(sqz_r):
    """given a squeezer parameter in dB, return output and relative y scale"""
    zeroreference = negative_test_dB_hacksqz(0).y
    out = negative_test_dB_hacksqz(sqz_r)
    return out, out.y/zeroreference

In [ ]:
# r values to plot
r23scaling = 20
# rT is the threshold value calculated before
r1, r2, r3, r4 = -rT, -rT/r23scaling, rT/r23scaling, rT
k1, k2, k3, k4 = dB_rel_hacksqz(r1), dB_rel_hacksqz(r2), dB_rel_hacksqz(r3), dB_rel_hacksqz(r4)

nk1 = negative_test_dB_rel_hacksqz(rT)
nk2 = negative_test_dB_rel_hacksqz(rT/r23scaling)

In [ ]:
# plotting r depedence
xax = k1[0].x

fig, ax = plt.subplots(figsize=(8,6))
ax.plot(xax, k4[1], label="rT")
ax.plot(xax, k1[1], label="-rT")
# ax.legend(*map(reversed, ax.get_legend_handles_labels()))
ax.legend(title="squeezer parameter, r / dB", loc="upper left", fancybox=True)
ax.set(title="relative homodyne intensity vs frequency offset",
       xlabel="frequency offset, $\Omega$ / Hz", ylabel="abs of relative homodyne intensity",
       ylim=(0, 2))
plt.show()

# values may not appear to diverge, however this comes down to choice of datapoints
# future James: I'm not sure what this is meant to say

In [ ]:
# plotting frequency dependence
ang_freq_ax = 2*np.pi*k1[0].x

analytics_data_freq = np.genfromtxt("squeezed_cavity_analytics_homodyne_intensity_vs_freq.csv",
                                    delimiter=",")
analytics_ang_freq_ax = analytics_data_freq[:,0]
num_lines = 12
linestyle_list = ["-","--","-","--","-","--","-","--","-.","-","--","-."]

fig, ax = plt.subplots(figsize=(10,8))
ax.set_prop_cycle(color=plt.cm.Spectral(np.linspace(0, 1, num_lines)), linestyle=linestyle_list)
ax.plot(ang_freq_ax, k4[1], label="Finesse: r = rT")
ax.plot(analytics_ang_freq_ax, analytics_data_freq[:,1], label="analytics: r = rT")
ax.plot(ang_freq_ax, k3[1], label="Finesse: r = rT/{}".format(r23scaling))
ax.plot(analytics_ang_freq_ax, analytics_data_freq[:,2], label="analytics: r = rT/20")
ax.plot(ang_freq_ax, dB_rel_hacksqz(0)[1], label="Finesse: r = 0")
ax.plot(analytics_ang_freq_ax, analytics_data_freq[:,3], label="analytics: r = 0")
ax.plot(ang_freq_ax, k2[1], label="Finesse: r = -rT/{}".format(r23scaling))
ax.plot(analytics_ang_freq_ax, analytics_data_freq[:,4], label="analytics: r = -rT/20")
ax.plot(ang_freq_ax, nk2[1], label="Finesse: r = rT/{}, $\phi$ = $\pi/2$".format(r23scaling))
ax.plot(ang_freq_ax, k1[1], label="Finesse: r = -rT")
ax.plot(analytics_ang_freq_ax, analytics_data_freq[:,5], label="analytics: r = -rT")
ax.plot(ang_freq_ax, nk1[1], label="Finesse: r = rT, $\phi$ = $\pi/2$")
# ax.legend(*map(reversed, ax.get_legend_handles_labels()))
ax.legend(loc="upper left", fancybox=True)
ax.set(xlabel="angular frequency offset, $\Omega$ / Hz", ylabel="abs of relative PSD of homo. intensity",
       xlim=(0, 5e9), ylim=(0.987, 1.013))
fig.tight_layout()
fig.savefig("squeezed_cavity_relative_qhd_vs_freq_comparison.pdf")
plt.show()

In [ ]:
# frequency dependance with squeezing turned off

analytics_data_empty = np.genfromtxt("empty_cavity_analytics_homodyne_intensity_vs_freq.csv",
                                          delimiter=",")
analytics_empty_ang_freq_ax = analytics_data_empty[:,0]

ax1_color = 'r'
ax2_color = 'b'
zero_variation_about = dB_hacksqz(0).y.mean()

fig, ax1 = plt.subplots(figsize=(10,8))
ax1.plot(ang_freq_ax, dB_hacksqz(0).y, ".", color=ax1_color)
ax1.set(xlabel="angular frequency offset, $\Omega$ / Hz",
        title="abs of non-relative PSD of homo. intens. for empty cavity\n",
        xlim=(0, 5e9), ylim=(zero_variation_about-1.5e-23, zero_variation_about+1.5e-23))
ax1.set_ylabel("Finesse: r = 0", color=ax1_color)
ax1.tick_params(axis='y', labelcolor=ax1_color)

ax2 = ax1.twinx()
ax2.plot(analytics_empty_ang_freq_ax, analytics_data_empty[:,1], ax2_color)
ax2.set_ylabel("analytics: r = 0", color=ax2_color)
ax2.tick_params(axis='y', labelcolor=ax2_color)

fig.tight_layout()
fig.savefig("empty_cavity_non-relative_qhd_vs_freq_comparison.pdf")
plt.show()

In [ ]:
# comparing threshold squeezer parameters in dB
print(10*np.log10(np.exp(0.026)),rT)

# comparing FSR's
print(2*np.pi*299792458/2*1e-9, ang_freq_ax[1000:][np.argmax(k4[1][1000:])]*1e-9)

In [ ]:
# testing the dB convention that nle is using
# adapted from Vaishali's test code to check nle installed
testcode = """
l l1 1 0 0 n1
bs bs1 0.5 0.5 0 0 n1 n2 n3 n4

s s0 1 ni na
nle nle1 {} 90 na nb
s s1 1 nb n4

qhd qn 180 n2 n3
fsig noise 1

xaxis noise f lin 1 10G 10000
"""

testkat_null = finesse.kat()
testkat_null.verbose = False
testkat_null.parse(testcode.format(0))
out_null = testkat_null.run()

null_reference_value = out_null.y[0,0]

In [ ]:
# function for finding some datapoints to plot
def testkat_gain(gain):
    """input: gain in dB as parameter to finesse's nle
    output: actual gain
    """

    testkat = finesse.kat()
    testkat.verbose = False
    testkat.parse(testcode.format(gain))
    out = testkat.run()

    return 20*np.log10(out.y[0,0]/null_reference_value)

In [ ]:
# nle behaviour is evidently to use 10*log10, see factor of two in plot
gain_axis = np.linspace(0, 10, 20)
qhd_axis = [testkat_gain(x) for x in gain_axis]

# recall that r = np.log(10**(gaindB/10))
# fig, ax = plt.subplots(figsize=(4,3))
# ax.plot(gain_axis, qhd_axis, '.')
# ax.set(xlabel='nle input value / "dB"',
#        title="testing Finesse squeezers: nle")
# ax.set_ylabel("$20\, \log_{10}(\mathrm{relative\; qhd})$ / dB", fontsize=14)

# plt.show()
# fig.savefig("testing_Finesse_squeezers-nle.pdf")

In [ ]:
# now do the same as above, but with sq, to compare to nle
kat = pykat.finesse.kat()
kat.verbose = 0

# adapted from Vaishali's test code to check nle installed
testcode = """
l l1 1 0 0 n1
bs bs1 0.5 0.5 0 0 n1 n2 n3 n4

s s0 1 ni na
sq sqz1 0 10 90 nb
s s1 1 nb n4

qhd qn 180 n2 n3
fsig noise 1

xaxis sqz1 r lin 0 10 100
"""

kat.parse(testcode)
out = kat.run()
# out.plot()

q = out['qn']
# fig, ax = plt.subplots(figsize=(4,3))
# ax.plot(out.x, 20*np.log10(q/q[0]), '--')
# ax.set(xlabel="sq input value / dB",
#        title="testing Finesse squeezers: sq")
# ax.set_ylabel("$20\, \log_{10}(\mathrm{relative\; qhd})$ / dB", fontsize=14)
# plt.show()
# fig.savefig("testing_Finesse_squeezers-sq.pdf")

In [ ]:
# overlaying the above plots to directly compare the behaviour

fig, ax = plt.subplots(figsize=(4,3))
ax.plot(gain_axis, qhd_axis, 'r.', label="nle")
ax.plot(out.x, 20*np.log10(q/q[0]), 'b--', label="sq")
ax.set(ylim=(0, None))
ax.set_xlabel("input value / \"dB\" or dB", fontsize=15)
ax.set_ylabel("$20\, \log_{10}(\mathrm{relative\; qhd})$ / dB", fontsize=15)
ax.tick_params(axis='both', labelsize=16)
plt.title("testing Finesse squeezers", fontsize=16)
ax.legend()

plt.show()
fig.savefig("testing_Finesse_squeezers_comparison.pdf")

In [ ]:
# troubleshooting analytics, consider an empty cavity
empty_kat = finesse.kat()
empty_kat.verbose = False

empty_kat.parse("""
l laser 1 0 0 n1 # input power = 1W
m m1 0.9 0.1 0 n1 n2
s s1 1 n2 n3 # L = 1m
m m2 1 0 0 n3 n4

#pd refl n1
ad refl 0 n1

xaxis laser f lin 0 1E9 1000 
yaxis abs:deg

put refl f $x1
""")

empty_out = empty_kat.run()
empty_out.plot()

In [ ]:
# retired functions

# def hacksqz(sqz_r):
#     """given a squeezer parameter, return output of Finesse"""
#     hackkat = deepcopy(hackbase)

#     # power dB corresponding to r??
#     converted_r = 10*np.log10(np.exp(sqz_r))
    
#     hackcode = """
#     nle nle1 {0} 0 nc2 nc3 # sqz angle = 0
#     """.format(converted_r)

#     hackkat.parse(hackcode)

#     return hackkat.run()

# def rel_hacksqz(sqz_r):
#     """given a squeezer parameter, return output and relative y scale"""
#     zeroreference = hacksqz(0).y
#     out = hacksqz(sqz_r)
#     return out, out.y/zeroreference


# these functions use the WRONG (or against convention) formula for dB, should be 20*log10(-)

# def r_to_gain_in_dB(r):
#     return 10*np.log10(np.exp(r))

# def gain_in_dB_to_r(gain):
#     return np.log(10**(gain/10))